In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv
import datetime
import time
#%%
# Main code begins at MAIN below. 
# The following function is the Gillespie simulation engine.

In [2]:
def ribosim(ratesmat, kadd_C, kadd_W, chainlength):
    '''
    inputs:
        ratesmat[i,j] = rate to transition from state j to i [=0 on diagonal]
        kadd_C, kadd_W = rates of the amino acid incorporation steps
        chainlength = total number of amino acids to incorporate 
    outputs:
        ts = times at which transitions occurred
        chain = amino acid chain (of length chainlength)
        states = list of states occupied during each dwell. states[i] is state exited at ts[i]. 
                 states[i+1] is state entered at ts[i].
    states: 
            0=ribosome A site empty
            1=binds C.GTP (correct)
            2=binds W.GTP (wrong)
            3=binds C.GDP (or -3 means that when this state exits, C amino acid is incorporated)
            4=binds W.GDP (or -4 means that when this state exits, W amino acid is incorporated)
    '''
    ## initialize
    nstate = len(ratesmat)        # number of states, ratesmat defined with R matrix below
    Nreport = 101                 # report progress every Nreport amino acids
    incorpprobc = kadd_C/(ratesmat[0,3]) # incorporation rate for correct amino acid, see below for kadd_C
    incorpprobw = kadd_W/(ratesmat[0,4]) # incorporation rate for wrong amino acid, see below for kadd_W
    t = 0.        # current time
    states = [0]  # initially in state 0, empty ribosome 
    currentstate = 0
    ts = []     # array to store transition times
    chain = ""  # string to record amino acid chain
    Rtot = np.sum(ratesmat,axis=0) # sum of rates along columns, which is total rate to go from state j to the 4 other states
    propens = ratesmat.copy()  # make a scratch copy that we can modify
    for j in range(nstate):  # normalize rates to get probabilities of transitions
        propens[:,j]=ratesmat[:,j]/Rtot[j]
    cumu = np.cumsum(propens,axis=0) # cumulumative sum along columns, will use to determine which transition occurred 
    wrongs = 0 # initialize number of wrong incorporated AA
    
### perform the Gillespie simulation
    while len(chain) < chainlength: 
        if len(chain) > Nreport: # progress reporting
            print('*')
            Nreport = Nreport + 100
        decide = np.random.random(3) # 3 decisions need to be made independently
        # first choose time of the next transition:
        tstep = - np.log(decide[0])/Rtot[currentstate] # drawing transition time from an exponential distribution 
                                                       # exponential dist of waiting times comes from Poisson process
        t += tstep               # time at which we exit currentstate 
        ts = ts + [t]            # append transition time to array
        # next decide type of the next transition:
        newstate = np.nonzero(cumu[:,currentstate]>decide[1])[0][0] # first True entry is our next state
        if currentstate==3 and newstate==0: # check if tRNA.GDP is bound, if so, decide whether to incorporate
            if decide[2] < incorpprobc: 
                chain = chain + "*"
                states[-1] = -1*states[-1] # flag this state as actually incorporating an amino by making it negative
                print("*", len(chain))
        if currentstate==4 and newstate==0: # check if tRNA.GDP is bound, if so, decide whether to incorporate
            if decide[2] < incorpprobw: 
                chain = chain + "X"
                wrongs +=1
                states[-1] = -1*states[-1] # flag this state as actually incorporating an amino by making it negative
                print("X", len(chain))
        currentstate = newstate
        states = states + [currentstate] # add newstate to array of states
    return (chain, ts, states, wrongs)

In [10]:
#%% MAIN
# set up simulation parameters
    
chainlength = 25 # terminate when amino acid chain gets this long
numchains = 10    # how many amino acid chains to generate

simtype = "HN" # determines model to simulate, HN for Hopfield-Ninio, Equil for equilibrium, real for realistic
# all rates below in 1/s, see Table 1 in text 

In [11]:
if simtype == "HN": 
    '''parameters for pure Hopfield-Ninio model. The only differences between correct
    and wrong amino acids are in the unbinding rates, and those differences are taken
    to be small to exaggerate the rate of wrong incorporations, for illustration.'''
    kadd_C = 0.005 # final incorporation step rate, C for correct, W for wrong
    phi_add = 1; kadd_W = kadd_C*phi_add
    
    # fixed parameters for ribosome cycle - correct amino acid; parameters based on in-vitro experimental measurements
    kc_on = 40    # k'_c
    kc_off = 0.5  # k_c
    lc_on = 1e-3   # l'_c very unlikely for GDP-bound tRNA-complex to bind 
    lc_off = 0.085  # l_c
    mhc = 0.01 # m'_c
    msc = 1e-3  # m_c very unlikely for for GTP synthesis to occur
    
    # fixed parameters for ribosome cycle - wrong amino acid
    phi1 =  1; kw_on = phi1*kc_on
    phim1 = 94; kw_off = phim1*kc_off  # W binds more weakly than C if phim1 > 1
    phim3 = 1; lw_on = phim3*lc_on
    phi2 =  1; mhw = phi2*mhc
    phim2 = 0.084; msw = phim2*msc
    phi3 =  7.9; lw_off = phi3*lc_off   # dictated from others by thermodynamic consistency, see Appendix A.1
    

elif simtype == "FDR": 
    ''' a model that discriminates mainly by forward rates '''
    kadd_C = 4.14 # final incorporation step rate, C for correct, W for wrong
    phi_add = 0.017; kadd_W = kadd_C*phi_add
    
    # fixed parameters for ribosome cycle - correct amino acid; parameters based on in-vitro experimental measurements
    kc_on = 40    # k'_c
    kc_off = 0.5  # k_c
    lc_on = 1e-3   # l'_c very unlikely for GDP-bound tRNA-complex to bind 
    lc_off = 0.085  # l_c
    mhc = 25 # m'_c
    msc = 1e-3  # m_c very unlikely for for GTP synthesis to occur
    
    # fixed parameters for ribosome cycle - wrong amino acid
    phi1 =  0.68; kw_on = phi1*kc_on
    phim1 = 1; kw_off = phim1*kc_off  # W binds more weakly than C if phim1 > 1
    phim3 = 0.0027; lw_on = phim3*lc_on
    phi2 =  0.048; mhw = phi2*mhc
    phim2 = 12.09; msw = phim2*msc
    phi3 =  1; lw_off = phi3*lc_off   # dictated from others by thermodynamic consistency, see Appendix A.1
       
elif simtype == "real": # use parameter values from Banerjee et al., www.pnas.org/cgi/content/short/1614838114
    '''parameters for realistic model. Forward and unbinding rates are now both different.'''
    kadd_C = 4.14 
    phi_add = 0.017; kadd_W = kadd_C*phi_add
    
    # fixed parameters for ribosome cycle - correct amino acid
    kc_on = 40    # k'_c
    kc_off = 0.5  # k_c
    lc_on = 1e-3   # l'_c very unlikely for GDP-bound tRNA-complex to bind  
    lc_off = 0.085  # l_c
    mhc = 25 # m'_c
    msc = 1e-3  # m_c very unlikely for for GTP synthesis to occur
    
    # fixed parameters for ribosome cycle - wrong amino acid
    phi1 =  0.68; kw_on = phi1*kc_on
    phim1 = 94; kw_off = phim1*kc_off  
    phi3 =  7.9; lw_off = phi3*lc_off
    phi2 =  0.048; mhw = phi2*mhc
    phim2 = 1; msw = phim2*msc
    phim3 =  0.0027; lw_on = phim3*lc_on # dictated from others by thermodynamic consistency, see Appendix A.1

In [12]:
# set up ratesmat matrix, which is called R below    
'''    states: 
            0=ribosome A site empty
            1=binds C.GTP (right)
            2=binds W.GTP (wrong)
            3=binds C.GDP
            4=binds W.GDP
'''

'    states: \n            0=ribosome A site empty\n            1=binds C.GTP (right)\n            2=binds W.GTP (wrong)\n            3=binds C.GDP\n            4=binds W.GDP\n'

In [13]:
R = np.zeros((5,5)) # R[i,j]=rate to transition from state j to i, or 0 if j=i
R[0,0] = 0.
R[1,0] = kc_on
R[2,0] = kw_on
R[3,0] = lc_on
R[4,0] = lw_on
#
R[0,1] = kc_off
R[1,1] = 0.
R[2,1] = 0.
R[3,1] = mhc
R[4,1] = 0.
#
R[0,2] = kw_off
R[1,2] = 0.
R[2,2] = 0.
R[3,2] = 0.
R[4,2] = mhw
#
R[0,3] = lc_off + kadd_C # total rate to unbind or incorporate
R[1,3] = msc
R[2,3] = 0.
R[3,3] = 0.
R[4,3] = 0.
#
R[0,4] = lw_off + kadd_W # total rate to unbind or incorporate
R[1,4] = 0.
R[2,4] = msw
R[3,4] = 0.
R[4,4] = 0.

In [14]:
#%% run Gillespie simulation with parameters defined above
print (R)
chains = [] # placeholder for a list with simulated chains
ts = []     # placeholder for a list of transition times for each chain
states = [] # placeholder for a list of states for each chain

[[0.000e+00 5.000e-01 4.700e+01 9.000e-02 6.765e-01]
 [4.000e+01 0.000e+00 0.000e+00 1.000e-03 0.000e+00]
 [4.000e+01 0.000e+00 0.000e+00 0.000e+00 8.400e-05]
 [1.000e-03 1.000e-02 0.000e+00 0.000e+00 0.000e+00]
 [1.000e-03 0.000e+00 1.000e-02 0.000e+00 0.000e+00]]


In [15]:
err = [ ]
for i in range(1,numchains+1):
    print("chain ", i)
    chainM, tsM, statesM, wrongs = ribosim(R, kadd_C, kadd_W, chainlength) # use M for each iteration, no M for multiple chains
    err.append(wrongs)
    chains = chains + [chainM]
    ts.append(tsM)
    states.append(statesM)
    print(ts[i-1][chainlength]," duration; ", wrongs," errors")

chain  1
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
* 25
11.804970490945022  duration;  0  errors
chain  2
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
* 25
15.988668880295764  duration;  0  errors
chain  3
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
* 25
34.25400252357954  duration;  0  errors
chain  4
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
* 25
13.346004029298516  duration;  0  errors
chain  5
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
X 25
19.114890815584765  duration;  1  errors
chain  6
* 1
* 2
* 3
* 4
* 5
* 6
* 7
* 8
* 9
* 10
* 11
* 12
* 13
* 14
* 15
* 16
* 17
* 18
* 19
* 20
* 21
* 22
* 23
* 24
* 25
31.296473453696137  duration;  0  errors
chain

In [16]:
chains = np.array(chains)
ts = np.array(ts)
states = np.array(states)
now = datetime.datetime.now() # date when the simulation was done
np.savez('kproof_'+str(simtype)+time.strftime("_%m%d"), wrongs=err, chains=chains, ts=ts, states=states, now=now, simtype=simtype, R=R) # save simulation data

from IPython.display import Audio
display(Audio('W:/University/Studying/israel/2 semester/Deep Learn/finish.mp3', autoplay=True))

In [ ]:
!pip install numpy==1.16.1

In [17]:
#%% view data from a previous simulation
data = np.load('kproof_real_1119.npz') # example data file name shown here
print("Model:", data['simtype'])
print("Numbers of wrong AAs:\t\t\t", "Final times:\n")
for i in range(0,len(data['chains'])):
    print(data['wrongs'][i], "\t\t\t\t\t", data['ts'][i][chainlength-1], "\n")

Model: real
Numbers of wrong AAs:			 Final times:

0 					 3.686909910556665 

0 					 1.5695134184973927 

0 					 1.5034895047798778 

0 					 2.0126265519091047 

0 					 1.9985197131148544 

0 					 1.1782187744163217 

0 					 1.9219995867524242 

0 					 1.4535197903046988 

0 					 1.0813984670623378 

0 					 1.4208522314518248 



In [19]:
data = np.load('kproof_FDR_1119.npz') # example data file name shown here
print("Model:", data['simtype'])
print("Numbers of wrong AAs:\t\t\t", "Final times:\n")
for i in range(0,len(data['chains'])):
    print(data['wrongs'][i], "\t\t\t\t\t", data['ts'][i][chainlength-1], "\n")

Model: FDR
Numbers of wrong AAs:			 Final times:

4 					 5.14645851194112 

7 					 11.054442422899465 

5 					 19.389706260448644 

3 					 7.107365861371472 

5 					 29.626253219151035 

3 					 3.888515576154654 

3 					 5.019653330189309 

2 					 13.007772659237347 

6 					 11.803525992303026 

2 					 6.122732986573573 



In [20]:
data = np.load('kproof_HN_1119.npz') # example data file name shown here
print("Model:", data['simtype'])
print("Numbers of wrong AAs:\t\t\t", "Final times:\n")
for i in range(0,len(data['chains'])):
    print(data['wrongs'][i], "\t\t\t\t\t", data['ts'][i][chainlength-1], "\n")

Model: HN
Numbers of wrong AAs:			 Final times:

0 					 11.786017246197298 

0 					 14.394717152681675 

0 					 34.252102727141136 

0 					 13.315090101559232 

1 					 19.11402421592814 

0 					 31.270798605738005 

0 					 59.669259882709326 

0 					 16.997306212016916 

0 					 41.180097624192044 

0 					 29.604198272644044 

